In [130]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import MinMaxScaler


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. Bus

In [131]:
bus = pd.read_csv('../data/clean_data/bus_clean.csv')

In [132]:
bus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49051 entries, 0 to 49050
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       49051 non-null  object
 1   route      49051 non-null  int64 
 2   time       49051 non-null  object
 3   day        49051 non-null  object
 4   location   49051 non-null  object
 5   incident   49051 non-null  object
 6   min_delay  49051 non-null  int64 
 7   min_gap    49051 non-null  int64 
 8   direction  49051 non-null  object
 9   vehicle    49051 non-null  int64 
 10  day_month  49051 non-null  int64 
 11  month      49051 non-null  int64 
 12  hour       49051 non-null  int64 
 13  min        49051 non-null  int64 
dtypes: int64(8), object(6)
memory usage: 5.2+ MB


## 1.2 StringIndexer col categorical

In [133]:
indexed_data = bus.copy()

In [134]:
categorical_cols = ["day", "incident", "direction", "location"]

In [135]:
le = LabelEncoder()
le_day = le.fit(indexed_data["day"].unique())
indexed_data["day_indexed"] = le_day.transform(indexed_data["day"])

le = LabelEncoder()
le_code = le.fit(indexed_data["incident"].unique())
indexed_data["incident_indexed"] = le_code.transform(indexed_data["incident"])

le = LabelEncoder()
le_bound = le.fit(indexed_data["direction"].unique())
indexed_data["direction_indexed"] = le_bound.transform(indexed_data["direction"])

le = LabelEncoder()
le_line = le.fit(indexed_data["location"].unique())
indexed_data["location_indexed"] = le_line.transform(indexed_data["location"]) 

## 2.3 W2v cal station

In [136]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(indexed_data['location']) 
sequences = tokenizer.texts_to_sequences(indexed_data['location'])
station_w2v = pad_sequences(sequences, maxlen=7) 

In [137]:
indexed_data['location_w2v'] = station_w2v.tolist()

## 2.4 Vector and scala feature col

In [138]:
feature_cols =['route',
 'min_gap',
 'vehicle',
 'day_month',
 'month',
 'hour',
 'min',
 'day_indexed',
 'incident_indexed',
 'direction_indexed']

In [139]:
features_data = indexed_data.copy()

In [140]:
selected_columns_df = features_data[feature_cols].values
features_data['features'] = features_data['location_w2v']

for i in range(len(selected_columns_df)):
    features_data.at[i, 'features'] = features_data.at[i, 'features'] + selected_columns_df[i].tolist()

In [141]:
array = np.array(features_data['features'].tolist())

scaler = MinMaxScaler()
scaler_model =scaler.fit(array)
scaled_array = scaler_model.transform(array)

features_data['scaled'] = scaled_array.tolist()

In [142]:
# index
feature_cols.append('location_indexed')
features_data['features_index'] = features_data[feature_cols].apply(lambda row: row.tolist(), axis=1)

In [143]:
array = np.array(features_data['features_index'].tolist())

scaler = MinMaxScaler()
scaler_model =scaler.fit(array)
scaled_array = scaler_model.transform(array)

features_data['scaled_index'] = scaled_array.tolist()

## 2.6 Save data featured

In [144]:
features_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49051 entries, 0 to 49050
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   date               49051 non-null  object
 1   route              49051 non-null  int64 
 2   time               49051 non-null  object
 3   day                49051 non-null  object
 4   location           49051 non-null  object
 5   incident           49051 non-null  object
 6   min_delay          49051 non-null  int64 
 7   min_gap            49051 non-null  int64 
 8   direction          49051 non-null  object
 9   vehicle            49051 non-null  int64 
 10  day_month          49051 non-null  int64 
 11  month              49051 non-null  int64 
 12  hour               49051 non-null  int64 
 13  min                49051 non-null  int64 
 14  day_indexed        49051 non-null  int32 
 15  incident_indexed   49051 non-null  int32 
 16  direction_indexed  49051 non-null  int32

In [145]:
features_data.to_csv('../data/features_data/bus_feature_pd.csv', index=False)

# 2. SubWay

## 2.1 Load data

In [168]:
subway = pd.read_csv('../data/clean_data/subway_cleaned.csv')

In [169]:
# subway_drop = subway.drop(["date", "time"], axis=1)
# subway_drop

## 2.2 StringIndexer col categorical

In [170]:
# categorical_cols = ["day", "code", "bound", "line"]

In [171]:
indexed_data = subway.copy()

In [172]:
indexed_data

,date,time,day,station,code,min_delay,min_gap,bound,line,vehicle,day_month,month,hour,min,at_station
0,2023-01-01,02:22:00,Sunday,museum station,MUPAA,3,9,S,YU,5931,1,1,2,22,1
1,2023-01-01,08:02:00,Sunday,vaughan mc station,TUNOA,6,12,S,YU,5451,1,1,8,2,1
2,2023-01-01,08:11:00,Sunday,greenwood station,MUNOA,5,10,E,BD,0,1,1,8,11,1
3,2023-01-01,08:15:00,Sunday,keele station,TUNOA,5,10,W,BD,0,1,1,8,15,1
4,2023-01-01,08:16:00,Sunday,greenwood station,TUNOA,5,10,E,BD,0,1,1,8,16,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7377,2023-11-30,22:37:00,Thursday,glencairn station,SUUT,14,19,S,YU,5761,30,11,22,37,1
7378,2023-11-30,23:09:00,Thursday,bloor yonge station,MUSAN,5,10,S,YU,5666,30,11,23,9,1
7379,2023-11-30,01:57:00,Thursday,sheppard yonge station,MUATC,5,10,N,YU,6066,30,11,1,57,1
7380,2023-11-30,06:29:00,Thursday,bessarion station,TUML,7,0,E,SHP,6186,30,11,6,29,1


In [173]:
le = LabelEncoder()
le_day = le.fit(indexed_data["day"].unique())
indexed_data["day_indexed"] = le_day.transform(indexed_data["day"])

le = LabelEncoder()
le_code = le.fit(indexed_data["code"].unique())
indexed_data["code_indexed"] = le_code.transform(indexed_data["code"])

le = LabelEncoder()
le_bound = le.fit(indexed_data["bound"].unique())
indexed_data["bound_indexed"] = le_bound.transform(indexed_data["bound"])

le = LabelEncoder()
le_line = le.fit(indexed_data["line"].unique())
indexed_data["line_indexed"] = le_line.transform(indexed_data["line"]) 

le = LabelEncoder()
le_station = le.fit(indexed_data["station"].unique())
indexed_data["station_indexed"] = le_station.transform(indexed_data["station"]) 

## 2.3 W2v cal station

In [174]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(indexed_data['station']) 
sequences = tokenizer.texts_to_sequences(indexed_data['station'])
station_w2v = pad_sequences(sequences, maxlen=4) 

In [175]:
indexed_data['station_w2v'] = station_w2v.tolist()

## 2.4 Vector and scala feature col

In [176]:
feature_cols =['min_gap',
 'vehicle',
 'day_month',
 'month',
 'hour',
 'min',
 'at_station',
 'day_indexed',
 'code_indexed',
 'bound_indexed',
 'line_indexed'
 ]

In [177]:
features_data = indexed_data.copy()

In [178]:
selected_columns_df = features_data[feature_cols].values

In [179]:
features_data['features'] = features_data['station_w2v']

In [180]:
for i in range(len(selected_columns_df)):
    features_data.at[i, 'features'] = features_data.at[i, 'features'] + selected_columns_df[i].tolist()

In [192]:
array = np.array(features_data['features'].tolist())

scaler = MinMaxScaler()
scaler_model =scaler.fit(array)
scaled_array = scaler_model.transform(array)

features_data['scaled'] = scaled_array.tolist()

In [182]:
# index
feature_cols.append('station_indexed')
features_data['features_index'] = features_data[feature_cols].apply(lambda row: row.tolist(), axis=1)

In [183]:
array = np.array(features_data['features_index'].tolist())

# Assuming X is your data
scaler = MinMaxScaler()
scaler_model =scaler.fit(array)
# Fit to data, then transform it
scaled_array = scaler_model.transform(array)

features_data['scaled_index'] = scaled_array.tolist()

## 2.6 Save data featured

In [184]:
features_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7382 entries, 0 to 7381
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             7382 non-null   object
 1   time             7382 non-null   object
 2   day              7382 non-null   object
 3   station          7382 non-null   object
 4   code             7382 non-null   object
 5   min_delay        7382 non-null   int64 
 6   min_gap          7382 non-null   int64 
 7   bound            7382 non-null   object
 8   line             7382 non-null   object
 9   vehicle          7382 non-null   int64 
 10  day_month        7382 non-null   int64 
 11  month            7382 non-null   int64 
 12  hour             7382 non-null   int64 
 13  min              7382 non-null   int64 
 14  at_station       7382 non-null   int64 
 15  day_indexed      7382 non-null   int32 
 16  code_indexed     7382 non-null   int32 
 17  bound_indexed    7382 non-null   

In [185]:
features_data.to_csv('../data/features_data/subway_feature_pd.csv', index=False)

# 3.

In [186]:
X =subway.loc[0]
X

date              2023-01-01
time                02:22:00
day                   Sunday
station       museum station
code                   MUPAA
min_delay                  3
min_gap                    9
bound                      S
line                      YU
vehicle                 5931
day_month                  1
month                      1
hour                       2
min                       22
at_station                 1
Name: 0, dtype: object

In [196]:
def feature(X):
    X =X.copy()
    X['day_indexed'] = le_day.transform([X['day']])[0]
    X['code_indexed'] = le_code.transform([X['code']])[0]
    X['bound_indexed'] = le_bound.transform([X['bound']])[0]
    X['line_indexed'] = le_line.transform([X['line']])[0]
    X['station_w2v'] = tokenizer.texts_to_sequences([X['station']])[0]
    X['station_w2v'] = pad_sequences([X['station_w2v']], maxlen=4)[0]
    selected_columns = [X['min_gap'], X['vehicle'], X['day_month'], X['month'], X['hour'], X['min'], X['at_station'], X['day_indexed'], X['code_indexed'], X['bound_indexed'], X['line_indexed']]
    X['features'] = X['station_w2v'].tolist() + selected_columns
    # print(X['features'])
    X['features'] = scaler_model.transform([X['features']])[0]
    return X['features']

In [197]:
len(feature(X))

15